<a href="https://colab.research.google.com/github/Kangjunseo/NLP/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q keras
import keras

In [0]:
from keras.models import Sequential
from keras.layers import SimpleRNN

In [11]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일함.
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [12]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (8, 3)                    42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [13]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (8, 2, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [0]:
text="나랑 점심 먹으러 갈래 메뉴는 햄버거 점심 메뉴 좋지"

In [0]:
from keras_preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts([text])
encoded = t.texts_to_sequences([text])[0]
# [0]을 해주지 않으면 [[contents]]와 같은 리스트 안의 리스트 형태로 저장 됨.
# [0]을 해주면 [contents]와 같은 하나의 리스트로 저장됨.

In [16]:
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 9


In [17]:
print(t.word_index)

{'점심': 1, '나랑': 2, '먹으러': 3, '갈래': 4, '메뉴는': 5, '햄버거': 6, '메뉴': 7, '좋지': 8}


In [18]:
sequences = list()
for c in range(1, len(encoded)):
    sequence = encoded[c-1:c+1] # 단어를 두개씩 묶어서 저장해준다. 이는 X와 Y의 관계를 구성하기 위함이다.
    sequences.append(sequence)
print('단어 묶음의 개수: %d' % len(sequences))

단어 묶음의 개수: 8


In [19]:
print(sequences)

[[2, 1], [1, 3], [3, 4], [4, 5], [5, 6], [6, 1], [1, 7], [7, 8]]


In [0]:
import numpy as np
X, y = zip(*sequences) # 첫번째 열이 X, 두번째 열이 y가 됨.
X=np.array(X) # 타입을 배열로 변환
y=np.array(y) # 타입을 배열로 변환

In [21]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes=vocab_size) # 원 핫 인코딩
print(y)

[[0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [0]:
from keras.layers import Embedding, Dense, SimpleRNN
from keras.models import Sequential
model = Sequential()
model.add(Embedding(vocab_size, 9, input_length=1))
# 단어 집합의 크기는 9. 임베딩 벡터의 크기는 9. 각 sample의 길이는 단어 한 개이므로 길이는 1.
model.add(SimpleRNN(9))
# RNN의 결과값으로 나오는 벡터의 차원 또한 9로 한다. 더 크게 해주어도 상관은 없음.
model.add(Dense(vocab_size, activation='softmax'))
# 출력층을 지나서 나오는 벡터의 크기도 9로 한다.

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=500, verbose=2)

In [24]:
print(t.word_index.items())

dict_items([('점심', 1), ('나랑', 2), ('먹으러', 3), ('갈래', 4), ('메뉴는', 5), ('햄버거', 6), ('메뉴', 7), ('좋지', 8)])


In [0]:
def predict_next_word(model, t, current_word): # 모델, 토크나이저, 현재 단어를 받아온다.
  encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
  encoded = np.array(encoded) # 현재 단어에 대한 정수 인코딩
  result = model.predict_classes(encoded, verbose=0)
  # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
  for word, index in t.word_index.items(): # 위에서 실습한 것처럼 단어와 인덱스를 리턴.
    if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
        return word # 그 단어를 출력

In [29]:
print(predict_next_word(model, t, '먹으러'))
print(predict_next_word(model, t, '점심'))
print(predict_next_word(model, t, '메뉴'))

갈래
먹으러
좋지


In [0]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = np.array(encoded) # 현재 단어에 대한 정수 인코딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = word # 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [31]:
print(sentence_generation(model, t, '먹으러', 6))

먹으러 갈래 메뉴는 햄버거 점심 먹으러 갈래


##문맥반영

In [0]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [0]:
from keras_preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts([text])
encoded = t.texts_to_sequences([text])[0]

In [35]:
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야하므로 미리 +1 선언 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [36]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [37]:
sequences = list()
for line in text.split('\n'): # Wn을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print('훈련 데이터의 개수: %d' % len(sequences))

훈련 데이터의 개수: 11


In [38]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


[2, 3]은 [경마장에, 있는]에 해당되며 [2, 3, 1]은 [경마장에, 있는, 말이]

In [0]:
from keras.preprocessing.sequence import pad_sequences
sequences = pad_sequences(sequences, maxlen=6, padding='pre')

In [40]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [42]:
import numpy as np
sequences = np.array(sequences)

X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 열을 제외하고 저장한 것은 X
# 리스트의 마지막 열만 저장한 것은 y
print(X,y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]] [ 3  1  4  5  1  7  1  9 10  1 11]


In [0]:
from keras.utils import to_categorical
y = to_categorical(y, num_classes=vocab_size)

In [44]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [45]:
from keras.layers import Embedding, Dense, SimpleRNN
from keras.models import Sequential

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=5)) # y를 제거하였으므로 이제 X의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
 - 0s - loss: 2.5126 - acc: 0.0000e+00
Epoch 2/200
 - 0s - loss: 2.4974 - acc: 0.0000e+00
Epoch 3/200
 - 0s - loss: 2.4827 - acc: 0.0909
Epoch 4/200
 - 0s - loss: 2.4685 - acc: 0.0909
Epoch 5/200
 - 0s - loss: 2.4546 - acc: 0.0909
Epoch 6/200
 - 0s - loss: 2.4408 - acc: 0.0909
Epoch 7/200
 - 0s - loss: 2.4270 - acc: 0.2727
Epoch 8/200
 - 0s - loss: 2.4131 - acc: 0.4545
Epoch 9/200
 - 0s - loss: 2.3989 - acc: 0.4545
Epoch 10/200
 - 0s - loss: 2.3844 - acc: 0.4545
Epoch 11/200
 - 0s - loss: 2.3695 - acc: 0.4545
Epoch 12/200
 - 0s - loss: 2.3541 - acc: 0.4545
Epoch 13/200
 - 0s - loss: 2.3380 - acc: 0.4545
Epoch 14/200
 - 0s - loss: 2.3212 - acc: 0.4545
Epoch 15/200
 - 0s - loss: 2.3037 - acc: 0.4545
Epoch 16/200
 - 0s - loss: 2.2854 - acc: 0.4545
Epoch 17/200
 - 0s - loss: 2.2661 - acc: 0.4545
Epoch 18/200
 - 0s - loss: 2.2460 - acc: 0.4545
Epoch 19/200
 - 0s - loss: 2.2250 - acc: 0.4545
Epoch 20/200
 - 0s - loss: 2.2029 - acc: 0.4545
Epoch 21/200
 - 0s - loss: 2.1800 - acc: 

In [0]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [48]:
print(sentence_generation(model, t, '경마장에', 4))
# '경마장에' 라는 단어 뒤에는 총 4개의 단어가 있으므로 4번 예측

말이 말이 법이다 말이 법이다


In [49]:
print(sentence_generation(model, t, '그의', 2)) # 2번 예측

그의 말이 법이다
